In [14]:
#pip install selenium
#pip install webdriver-manager
#pip install free-proxy

Defaulting to user installation because normal site-packages is not writeable
  Using cached selenium-4.18.1-py3-none-any.whl (10.0 MB)
  Using cached trio-0.25.0-py3-none-any.whl (467 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
  Using cached exceptiongroup-1.2.0-py3-none-any.whl (16 kB)
  Using cached attrs-23.2.0-py3-none-any.whl (60 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: attrs
    Found existing installation: attrs 22.2.0
    Uninstalling attrs-22.2.0:
      Successfully uninstalled attrs-22.2.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.5.0
    Uninstalling typing-extensions-4.5.0:
      Successfully uninstalled typing-extensions-4.5.0
You should consider upgrading via the '/Applications/Xcode.app/Co

In [19]:
import queue
from threading import Thread
import requests
from bs4 import BeautifulSoup

class FreeProxy:
    def __init__(self):
        self.headers = {
            "Accept": "*/*",
            "Accept-Language": "en-US,en;q=0.5",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
        }


    def get_proxies(self):
        self.proxies = queue.Queue()
        
        url = "https://free-proxy-list.net"
        r = requests.get(url, headers=self.headers)

        soup = BeautifulSoup(r.text, "html.parser")
        table_rows = soup.find("section", attrs={"id": "list"}).find("tbody").find_all("tr")

        for row in table_rows:
            ip_address = row.find_all("td")[0].text.strip()
            port = row.find_all("td")[1].text.strip()

            if row.find_all("td")[6].text.strip() == "yes":
                self.proxies.put(f"{ip_address}:{port}")


    def check_proxies(self):
        self.valid_proxies = []

        while not self.proxies.empty():
            proxy = self.proxies.get()

            try:
                r = requests.get("https://www.google.com", headers=self.headers, proxies={"http": proxy, "https": proxy}, timeout=5)
            except:
                continue

            if r.status_code == 200:
                self.valid_proxies.append(proxy)


    def get_proxy_list(self):
        while True:
            try:
                self.get_proxies()

                threads = [Thread(target=self.check_proxies) for _ in range(32)]

                [t.start() for t in threads]
                [t.join() for t in threads]

                assert len(self.valid_proxies) > 0
            
                return self.valid_proxies
            except:
                continue
    

if __name__ == "__main__":
    free_proxy = FreeProxy()
    proxy_list = free_proxy.get_proxy_list()
    print(proxy_list)

['35.185.196.38:3128']


In [22]:
import time
import random 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

free_proxy = FreeProxy() 
print("Getting proxy list...")

proxy_list = free_proxy.get_proxy_list()
print(proxy_list)

for _ in range(2):
    random_proxy = random.choice(proxy_list)
    print(random_proxy)

    chrome_options = Options()
    chrome_options.add_argument(f"--proxy-server={random_proxy}")
    chrome_options.add_argument("--lang=en-US")
    chrome_options.add_argument("--start-maximized")

    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

    try:
        driver.get("https://www.portalecreditori.it/procedure.php?filter=txtFAnno%7C2021")

        time.sleep(5)
        driver.close()
    except:
        time.sleep(5)
        driver.close()

Getting proxy list...
['4.144.161.159:80', '35.185.196.38:3128']
35.185.196.38:3128
35.185.196.38:3128


In [36]:
from fp.fp import FreeProxy
from selenium import webdriver
import selenium.webdriver as webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager
import random

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# randomly extract a proxy
random_proxy = FreeProxy(country_id=['US']).get()

# Set up ChromeOptions
chrome_options = ChromeOptions()

# Set proxy
chrome_options.add_argument(f'--proxy-server={random_proxy}')

# Set up ChromeService
chrome_service = ChromeService(ChromeDriverManager().install())

# create a Chromedriver instance
driver = webdriver.Chrome(
    service=chrome_service,
    #options=chrome_options
)

In [37]:
driver.get("https://www.portalecreditori.it/procedure.php?filter=txtFAnno%7C2021")

In [46]:
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Wait for the table element to be present
table = WebDriverWait(driver, 100).until(
    EC.presence_of_element_located((By.XPATH, "//table[@id='elenco']"))
)

# Extract table headers
headers = []
header_elements = table.find_elements(By.XPATH, ".//th")
for header_element in header_elements:
    headers.append(header_element.text)

# Extract table rows
rows = []
row_elements = table.find_elements(By.XPATH, ".//tbody//tr")
for row_element in row_elements:
    row = []
    cell_elements = row_element.find_elements(By.XPATH, ".//td")
    for index, cell_element in enumerate(cell_elements):
        if index == 0:  # Check if it's the first column
                link = cell_element.find_element(By.XPATH, ".//a").get_attribute("href")
                row.append(cell_element.text) 
        else:
            row.append(cell_element.text)
    row.append(link)  # Append the hyperlink to the row
    rows.append(row)

headers.extend(["Link"])  # Append "Link" to the headers

print(headers)
# Create DataFrame
df = pd.DataFrame(rows, columns=headers)

# remove problemeatic first row
df = df.iloc[1:]

# Iterate through each row to visit the hyperlinks and extract information from the vertical tables
for index, row in df.iterrows():
    # Visit the hyperlink
    driver.get(row["Link"])
    
    # Add some delay to avoid being blocked
    time.sleep(random.uniform(1, 3))  # Adjust the delay range as needed
    
    # Find the table with class 'table-vertical-header' and extract information from it
    vertical_table = driver.find_element(By.CLASS_NAME, "table-vertical-header")
    
    # Extracting data from the vertical table
    data = []
    header_elements = vertical_table.find_elements(By.TAG_NAME, "th")
    for header_element in header_elements:
        data.append(header_element.text)
    row_elements = vertical_table.find_elements(By.TAG_NAME, "td")
    for row_element in row_elements:
        data.append(row_element.text)
    
    # You can store or process the extracted data as needed
    print(data)  # Example: Printing the data for demonstration

# Close the browser
driver.quit()


TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x00000001033132a8 chromedriver + 4625064
1   chromedriver                        0x000000010330ae53 chromedriver + 4591187
2   chromedriver                        0x0000000102f09e6a chromedriver + 392810
3   chromedriver                        0x0000000102f5541d chromedriver + 701469
4   chromedriver                        0x0000000102f555b1 chromedriver + 701873
5   chromedriver                        0x0000000102f991c4 chromedriver + 979396
6   chromedriver                        0x0000000102f7789d chromedriver + 841885
7   chromedriver                        0x0000000102f9668f chromedriver + 968335
8   chromedriver                        0x0000000102f77613 chromedriver + 841235
9   chromedriver                        0x0000000102f483da chromedriver + 648154
10  chromedriver                        0x0000000102f48d1e chromedriver + 650526
11  chromedriver                        0x00000001032d3a70 chromedr

In [45]:
df

,Procedura,PEC,Num./Anno\nData dich.,Tipo,Tribunale,Professionista,Giudice Delegato,Doc.,Ud. verif. S.P.,Decorrenza\ntardive,Virtual data room,Link
0,https://www.portalecreditori.it/procedura?id=J...,None,None,None,None,None,None,None,None,None,None,None
1,SDF LE AQUILE VGS INV VGS SEC GSV,Vedi,33/2021\n03/10/2023,F,Foggia,Orsino Roberta,Lazzara Caterina,-,28/03/2024\n10:00,25/10/2023,,https://www.portalecreditori.it/procedura?id=R...
2,DEA SRL,,6/2021\n06/09/2023,F,Paola,De Caro Valerie Stella,Torretta Matteo,-,Vedi,,,https://www.portalecreditori.it/procedura?id=n...
3,S.B.M. Sas di Mauro Foschi Srl e C. e del Soci...,Vedi,52/2021\n17/02/2022,F,Rimini,ASTORRE MANCINI,Miconi Francesca,-,Vedi,13/04/2022,,https://www.portalecreditori.it/procedura?id=a...
4,Fervorari Rottami s.r.l. in liquidazione,Vedi,197/2021\n31/12/2021,F,Brescia,Monte Vincenzo,Bruno Simonetta,1,Vedi,08/02/2022,,https://www.portalecreditori.it/procedura?id=q...
5,DUE A Calzature S.r.l.,Vedi,814/2021\n30/12/2021,F,Milano,Molgora Alberto,De Simone Laura,-,Vedi,26/02/2022,,https://www.portalecreditori.it/procedura?id=Q...
6,CAESAP Soc. Coop. in liquidazione,Vedi,84/2021\n30/12/2021,F,Salerno,Brigante Marta,Sicilia Francesca,1,Vedi,23/02/2022,,https://www.portalecreditori.it/procedura?id=P...
7,AGROSERVIZI FUTURA SRL,Vedi,79/2021\n30/12/2021,F,Parma,Mora Monica,Vernizzi Enrico,1,Vedi,29/03/2022,,https://www.portalecreditori.it/procedura?id=V...
8,4 MOTORSPORT GLOBAL SRL IN LIQUIDAZIONE,Vedi,78/2021\n30/12/2021,F,Parma,Martellotti Angelo,Vernizzi Enrico,1,Vedi,30/03/2022,,https://www.portalecreditori.it/procedura?id=e...
9,F.LLI PINCOLINI Srl,Vedi,77/2021\n30/12/2021,F,Parma,Sarti Ferdinando,Vernizzi Enrico,1,Vedi,30/03/2022,,https://www.portalecreditori.it/procedura?id=v...


In [11]:

# Iterate through the remaining pages
for page_num in range(2, 3):
    print(f"Scraping page {page_num}")
    # Construct the URL for the current page
    url = f"https://www.portalecreditori.it/procedure.php?filter=txtFAnno%7C2021&order=procedura&verso=asc&page={page_num}"
    # Load the page
    driver.get(url)
    
    # Wait for the table element to be present
    table = WebDriverWait(driver, 3000).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='elenco']"))
    )
    
    print("Table found")

    # Extract table rows
    rows = []
    row_elements = table.find_elements(By.XPATH, ".//tbody//tr")
    for row_element in row_elements:
        row = [cell.text for cell in row_element.find_elements(By.XPATH, ".//td")]
        rows.append(row)

    print("Rows extracted")
    
    # Append rows to DataFrame
    df = pd.concat([df, pd.DataFrame(rows, columns=headers)], ignore_index=True)

    print("Rows appended")

# Close the WebDriver
driver.quit()
    
# Display the DataFrame
print(df)


Scraping page 2


KeyboardInterrupt: 